# 🏡 Linear Regression Architecture Workshop

## Introduction

Welcome to the **Linear Regression Architecture Workshop**.  
This workshop is designed for college-level students learning both:

1. **Univariate Linear Regression** – a foundational algorithm in Machine Learning, focusing on predicting continuous values from a single feature.  
2. **Machine Learning Operations (MLOps)** – design patterns and architectural considerations that make machine learning experiments reproducible, scalable, and production-ready.  

We will use **real-world housing price data** from **California (USA)** and **Ontario (Canada)** as our case study.  
The goal is to not only understand how Linear Regression works, but also how to **design and implement a machine learning project** from sourcing data → building models → structuring code → preparing for deployment.  

The workshop will be completed in **two 2-hour sessions**, with **homework assignments** to be completed before each class.  

---

## Workshop Structure

### 📚 Session 1 – Univariate Linear Regression
- **Lecture focus**: Mathematical intuition, model formulation, gradient descent, cost function, evaluation metrics.  
- **Practical focus**: Implementing Univariate Linear Regression from scratch + using `scikit-learn`.  
- **Homework before class**: Data sourcing (from CSV, APIs, and relational databases).  

### ⚙️ Session 2 – Machine Learning Operations (MLOps)
- **Lecture focus**: Code modularity, reproducibility, experiment tracking, design patterns in ML architecture.  
- **Practical focus**: Architecting the project with pipelines, config management, and modular scripts.  
- **Homework before class**: Refactor previous Linear Regression code into modular, production-ready format.  

---

## Instructions for Students

### 🔹 Before Session 1: Data Sourcing

Your first task is to collect **housing price data** for California and Ontario.  
You must experiment with **at least three different types of data sources**:

1. **CSV Files**  
   - Find open housing datasets (e.g., Kaggle, UCI ML Repository, government portals).  
   - Example: [California Housing Dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset).  
   - Save datasets in `data/raw/` folder.  

2. **Web Services (APIs)**  
   - Explore free APIs offering housing, rental, or real-estate data.  
   - Example APIs:  
     - [Zillow (unofficial APIs exist, check docs)]  
     - [Realtor.ca data endpoints]  
     - [City of Toronto Open Data API](https://open.toronto.ca/)  
     - [California State Open Data Portal](https://data.ca.gov/).  
   - Use Python packages like `requests` or `httpx` to fetch data.  
   - Save results into structured JSON or convert to DataFrames.  

3. **Relational Databases**  
   - Connect to a **PostgreSQL** or **MySQL** demo database.  
   - Option 1: Use hosted databases with sample housing/economic data.  
   - Option 2: Load CSVs into a local database (e.g., PostgreSQL with `psql` or SQLite for portability).  
   - Connect from Python using `sqlalchemy` or `psycopg2`.  
   - Run SQL queries to filter/select data.  

💡 **Deliverable before Session 1**:  
- A Jupyter Notebook that loads housing price data from all three sources (CSV, API, Database) and explores it with basic descriptive statistics and plots.  

---

### 🔹 During Session 1: Univariate Linear Regression Experiment

1. **Define the Problem**  
   - Select one feature (e.g., median income, number of rooms, lot size) to predict housing price.  

2. **Preprocess Data**  
   - Handle missing values.  
   - Normalize/standardize features.  
   - Split data into **train/test sets**.  

3. **Model Implementation**  
   - Implement Linear Regression **from scratch**:  
     - Hypothesis function $ h_\theta(x) = \theta_0 + \theta_1 x $  
     - Cost function (MSE)  
     - Gradient descent update rule  
   - Implement Linear Regression **using scikit-learn** for comparison.  

4. **Model Evaluation**  
   - Compute RMSE, MAE, and $ R^2 $ score.  
   - Visualize regression line vs. data points.  

💡 **Deliverable during Session 1**:  
- A working notebook with both a manual and `scikit-learn` Linear Regression implementation.  

---

# Feature Selection

MeanValue and MaxValue was selected as the independent variable and WorkPeriod as the dependent variable.
The number of rooms is a fundamental housing characteristic with a clear and interpretable relationship to price. Using this feature supports the assumptions of univariate linear regression and keeps the model simple and easy to explain.

# Preprocess Data

Using the force values measured on each robot axis, I will analyze the robot’s time-series data, which consists of work periods and rest periods. The goal is to segment the data into work/rest periods based on the force signal. Each identified work period is assigned a sequential index, and this work-period number is used as the independent variable. For each work period, the mean force and peak force within that period are computed as the dependent variables.

Next, I group the work periods into detection intervals, where each interval contains 10 consecutive work periods. Within each detection interval, I run regression analysis on the dependent variables over the work-period index:

If the mean force shows a statistically significant upward trend, it suggests the robot may be experiencing system aging or gradual degradation.

If the peak force shows a statistically significant upward trend, it suggests the robot may be at risk of an urgent or imminent failure.

The raw dataset is located at:
data/raw/RMBR4-2_export_test.csv

I will transform it into a summarized table with the following fields:

work_period

mean_value

peak_value

interval_start_time (start time of the first work period in the interval)

interval_end_time (end time of the last work period in the interval)

Finally, I will export the resulting table to:
data/raw/RMBR4-2_export_test_1.csv

In [3]:
import os
import numpy as np
import pandas as pd

# =========================
# Config
# =========================
INPUT_PATH  = "data/raw/RMBR4-2_export_test.csv"
OUTPUT_PATH = "data/raw/RMBR4-2_export_test_1.csv"

SMOOTH_SECONDS      = 6.0
MAX_GAP_MULTIPLIER  = 10.0
MIN_GAP_SECONDS     = 6.0
MIN_WORK_SECONDS    = 8.0
THRESH_QUANTILE     = 0.65
THRESH_SCALE        = 0.5

# =========================
# Helpers
# =========================
def standardize_columns(frame: pd.DataFrame) -> pd.DataFrame:
    frame = frame.copy()
    frame.columns = (
        frame.columns.str.strip()
                     .str.lower()
                     .str.replace(" ", "_", regex=False)
                     .str.replace("#", "", regex=False)
    )
    return frame

def pick_time_column(columns) -> str:
    candidates = ["time", "recorded_at", "timestamp", "datetime"]
    for c in candidates:
        if c in columns:
            return c
    raise ValueError(f"Cannot find a time column. Available columns: {list(columns)}")

def compute_overall_force(frame: pd.DataFrame, axis_cols: list[str]) -> pd.Series:
    """
    Overall force per record.
    Default: mean(abs(axis_i)) across axes.
    """
    return frame[axis_cols].abs().mean(axis=1)

def merge_short_false_gaps(mask: np.ndarray, gap_n: int) -> np.ndarray:
    out = mask.copy()
    n = len(out)
    i = 0
    while i < n:
        if not out[i]:
            j = i
            while j < n and (not out[j]):
                j += 1
            left_true  = (i - 1 >= 0 and out[i - 1])
            right_true = (j < n and out[j])
            if left_true and right_true and (j - i) <= gap_n:
                out[i:j] = True
            i = j
        else:
            i += 1
    return out

def remove_short_true_runs(mask: np.ndarray, min_n: int) -> np.ndarray:
    out = mask.copy()
    n = len(out)
    i = 0
    while i < n:
        if out[i]:
            j = i
            while j < n and out[j]:
                j += 1
            if (j - i) < min_n:
                out[i:j] = False
            i = j
        else:
            i += 1
    return out

def label_true_runs(mask: np.ndarray) -> np.ndarray:
    labels = np.zeros(len(mask), dtype=int)
    run_id = 0
    i = 0
    n = len(mask)
    while i < n:
        if mask[i]:
            run_id += 1
            j = i
            while j < n and mask[j]:
                j += 1
            labels[i:j] = run_id
            i = j
        else:
            i += 1
    return labels

# =========================
# Load
# =========================
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input file not found: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH)
df = standardize_columns(df)

time_col = pick_time_column(df.columns)

axis_cols = [c for c in df.columns if c.startswith("axis_")]
if not axis_cols:
    raise ValueError(f"Cannot find axis columns. Columns: {df.columns.tolist()}")

# Drop axes that are entirely NaN, and sort
axis_cols = [c for c in axis_cols if df[c].notna().any()]
axis_cols = sorted(axis_cols, key=lambda x: int(x.split("_")[1]))

# Parse time and sort
df[time_col] = pd.to_datetime(df[time_col], errors="coerce", utc=True)
df = df.dropna(subset=[time_col]).sort_values(time_col).reset_index(drop=True)

# Estimate sampling interval
dt_sec = df[time_col].diff().dt.total_seconds()
median_dt = float(dt_sec.dropna().median()) if dt_sec.dropna().size else 1.0
if not np.isfinite(median_dt) or median_dt <= 0:
    median_dt = 1.0

# Compute overall force
force = compute_overall_force(df, axis_cols)

# Smooth force
win = max(3, int(np.ceil(SMOOTH_SECONDS / median_dt)))
force_smooth = force.rolling(win, min_periods=1).mean()

# Adaptive threshold
qv = float(np.nanpercentile(force_smooth, THRESH_QUANTILE * 100))
thr = 0.05 if qv <= 0 else THRESH_SCALE * qv

is_work = (force_smooth > thr).to_numpy()

# Break across big gaps
max_gap_seconds = MAX_GAP_MULTIPLIER * median_dt
big_gap = (dt_sec.fillna(0) > max_gap_seconds).to_numpy()
is_work = is_work & (~big_gap)

# Merge short rest gaps and remove short work segments
gap_n = max(1, int(np.ceil(MIN_GAP_SECONDS / median_dt)))
min_n = max(2, int(np.ceil(MIN_WORK_SECONDS / median_dt)))

is_work = merge_short_false_gaps(is_work, gap_n=gap_n)
is_work = remove_short_true_runs(is_work, min_n=min_n)

# Label work periods
work_labels = label_true_runs(is_work)
df["work_period"] = work_labels
df["overall_force"] = force

# Keep only work rows
df_work = df[df["work_period"] > 0].copy()
if df_work.empty:
    raise ValueError("No work periods detected. Try lowering THRESH_SCALE or THRESH_QUANTILE.")

# Summarize per work period (period-level times)
summary = (
    df_work.groupby("work_period")
           .agg(
               mean_value=("overall_force", "mean"),
               peak_value=("overall_force", "max"),
               period_start_time=(time_col, "min"),
               period_end_time=(time_col, "max"),
           )
           .reset_index()
)

# Save
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)
summary.to_csv(OUTPUT_PATH, index=False)

print(f"Saved: {OUTPUT_PATH}")
print(summary.head(15))

Saved: data/raw/RMBR4-2_export_test_1.csv
    work_period  mean_value  peak_value                period_start_time  \
0             1    3.302866    8.908955 2022-10-17 12:19:22.005000+00:00   
1             2    3.449832    6.300471 2022-10-17 12:20:56.771000+00:00   
2             3    3.491427    6.707684 2022-10-17 12:21:46.588000+00:00   
3             4    3.255471    5.874910 2022-10-17 12:22:45.266000+00:00   
4             5    2.881057    6.632019 2022-10-17 12:23:49.839000+00:00   
5             6    2.839395    5.545404 2022-10-17 12:24:39.488000+00:00   
6             7    3.030460    5.872156 2022-10-17 12:25:49.703000+00:00   
7             8    2.643483    5.146125 2022-10-17 12:26:43.029000+00:00   
8             9    3.208116    7.015793 2022-10-17 12:34:07.501000+00:00   
9            10    3.465327    5.787941 2022-10-17 12:34:58.709000+00:00   
10           11    3.586541    8.039739 2022-10-17 12:35:51.838000+00:00   
11           12    3.764583    7.634454 2022-1

In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# =========================
# Config
# =========================
INPUT_PATH = "data/raw/RMBR4-2_export_test_1.csv"
OUTPUT_DIR = "data/processed"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "RMBR4-2_export_processed.csv")

INTERVAL_SIZE = 10           # 10 work periods per detection interval
TEST_SIZE = 0.2              # last 20% as test (time-aware split)
FILL_METHOD = "median"       # options: "median", "mean", "ffill"

FEATURE_COLS = ["mean_value", "peak_value"]
TIME_COLS = ["period_start_time", "period_end_time"]

# =========================
# Load
# =========================
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input file not found: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH)

# =========================
# Validate columns
# =========================
required_cols = ["work_period"] + FEATURE_COLS + TIME_COLS
missing_cols = [c for c in required_cols if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}. Available: {df.columns.tolist()}")

# =========================
# Parse times and sort (time-aware)
# =========================
for c in TIME_COLS:
    df[c] = pd.to_datetime(df[c], errors="coerce", utc=True)

# Drop rows with invalid end time (critical)
df = df.dropna(subset=["period_end_time"]).copy()

# Ensure numeric
df["work_period"] = pd.to_numeric(df["work_period"], errors="coerce")
for c in FEATURE_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["work_period"] + FEATURE_COLS).copy()

# Sort by period end time (or you can sort by work_period)
df = df.sort_values("period_end_time").reset_index(drop=True)

# =========================
# Create interval_id (every 10 work periods)
# =========================
df["interval_id"] = ((df["work_period"] - 1) // INTERVAL_SIZE) + 1

# =========================
# Handle missing values (features)
# =========================
if FILL_METHOD == "ffill":
    df[FEATURE_COLS] = df[FEATURE_COLS].fillna(method="ffill").fillna(method="bfill")
elif FILL_METHOD == "mean":
    df[FEATURE_COLS] = df[FEATURE_COLS].fillna(df[FEATURE_COLS].mean(numeric_only=True))
else:  # median
    df[FEATURE_COLS] = df[FEATURE_COLS].fillna(df[FEATURE_COLS].median(numeric_only=True))

if df[FEATURE_COLS].isna().any().any():
    raise ValueError("Missing values remain after filling. Check your input data.")

# =========================
# Train/Test split (time-aware)
# =========================
n = len(df)
if n < 10:
    raise ValueError(f"Not enough periods ({n}) to split reliably.")

split_idx = int(np.floor((1 - TEST_SIZE) * n))
train_df = df.iloc[:split_idx].copy()
test_df  = df.iloc[split_idx:].copy()

# =========================
# Standardize features (fit on train, transform both)
# =========================
scaler = StandardScaler()
train_scaled = scaler.fit_transform(train_df[FEATURE_COLS])
test_scaled  = scaler.transform(test_df[FEATURE_COLS])

for i, col in enumerate(FEATURE_COLS):
    train_df[f"{col}_z"] = train_scaled[:, i]
    test_df[f"{col}_z"]  = test_scaled[:, i]

# Combine back
processed_df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)
processed_df["split"] = ["train"] * len(train_df) + ["test"] * len(test_df)

# =========================
# Save
# =========================
os.makedirs(OUTPUT_DIR, exist_ok=True)
processed_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved: {OUTPUT_PATH}")
print(f"Train size: {len(train_df)} | Test size: {len(test_df)}")
print(processed_df.head(15))

Saved: data/processed\RMBR4-2_export_processed.csv
Train size: 686 | Test size: 172
    work_period  mean_value  peak_value                period_start_time  \
0             1    3.302866    8.908955 2022-10-17 12:19:22.005000+00:00   
1             2    3.449832    6.300471 2022-10-17 12:20:56.771000+00:00   
2             3    3.491427    6.707684 2022-10-17 12:21:46.588000+00:00   
3             4    3.255471    5.874910 2022-10-17 12:22:45.266000+00:00   
4             5    2.881057    6.632019 2022-10-17 12:23:49.839000+00:00   
5             6    2.839395    5.545404 2022-10-17 12:24:39.488000+00:00   
6             7    3.030460    5.872156 2022-10-17 12:25:49.703000+00:00   
7             8    2.643483    5.146125 2022-10-17 12:26:43.029000+00:00   
8             9    3.208116    7.015793 2022-10-17 12:34:07.501000+00:00   
9            10    3.465327    5.787941 2022-10-17 12:34:58.709000+00:00   
10           11    3.586541    8.039739 2022-10-17 12:35:51.838000+00:00   
11  

# Model Implementation

Input: data/preprocessed/RMBR4-2_export_preprocessed.csv

For each detection interval (interval_id), perform linear regression using:

Independent variable (X): work_period

Dependent variables (y): mean_value_z and peak_value_z

Apply two implementations for comparison:

From scratch (gradient descent): estimate theta0 and theta1

scikit-learn: estimate theta0 and theta1

Aggregate the results into a new table keyed by interval_id, with one row per interval.

Output: data/models/interval_theta_table.csv

Note: Each interval yields two sets of parameters (mean and peak), producing eight parameter columns:

scratch_mean_theta0, scratch_mean_theta1

scratch_peak_theta0, scratch_peak_theta1

sklearn_mean_theta0, sklearn_mean_theta1

sklearn_peak_theta0, sklearn_peak_theta1

In [8]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# =========================
# Config
# =========================
INPUT_PATH = "data/processed/RMBR4-2_export_processed.csv"
OUTPUT_DIR = "data/models"
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "interval_theta_table_model_implement.csv")

X_COL = "work_period"
GROUP_COL = "interval_id"

# Use standardized targets from preprocessing
MEAN_TARGET = "mean_value_z"
PEAK_TARGET = "peak_value_z"

LEARNING_RATE = 0.05
ITERATIONS = 3000

# =========================
# From-scratch linear regression (1D) with gradient descent
# =========================
def hypothesis(theta0: float, theta1: float, x: np.ndarray) -> np.ndarray:
    """h_theta(x) = theta0 + theta1 * x"""
    return theta0 + theta1 * x

def mse_cost(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Mean squared error (MSE)"""
    return float(np.mean((y_true - y_pred) ** 2))

def gradient_descent_1d(x: np.ndarray, y: np.ndarray, lr: float, iters: int):
    """
    Train theta0, theta1 using gradient descent on 1D linear regression.

    Cost: J = (1/m) * sum((h(x_i) - y_i)^2)

    Gradients:
        dJ/dtheta0 = (2/m) * sum(h(x_i) - y_i)
        dJ/dtheta1 = (2/m) * sum((h(x_i) - y_i) * x_i)

    Updates:
        theta0 := theta0 - lr * dJ/dtheta0
        theta1 := theta1 - lr * dJ/dtheta1
    """
    x = x.astype(float)
    y = y.astype(float)

    m = len(x)
    if m == 0:
        raise ValueError("Empty training set for this interval.")

    theta0 = 0.0
    theta1 = 0.0

    for _ in range(iters):
        y_pred = hypothesis(theta0, theta1, x)
        error = y_pred - y

        grad_theta0 = (2.0 / m) * np.sum(error)
        grad_theta1 = (2.0 / m) * np.sum(error * x)

        theta0 -= lr * grad_theta0
        theta1 -= lr * grad_theta1

    return float(theta0), float(theta1)

def fit_scratch_with_centering(x: np.ndarray, y: np.ndarray, lr: float, iters: int):
    """
    Optional x-centering improves GD stability.
    Convert parameters back to original x scale after training.
    """
    x = x.astype(float)
    y = y.astype(float)

    mean_x = float(np.mean(x))
    x_center = x - mean_x

    theta0_c, theta1_c = gradient_descent_1d(x_center, y, lr=lr, iters=iters)

    # y = theta0_c + theta1_c*(x - mean_x) = (theta0_c - theta1_c*mean_x) + theta1_c*x
    theta0 = theta0_c - theta1_c * mean_x
    theta1 = theta1_c

    return float(theta0), float(theta1)

# =========================
# scikit-learn fit
# =========================
def fit_sklearn(x: np.ndarray, y: np.ndarray):
    model = LinearRegression()
    model.fit(x.reshape(-1, 1), y)
    return float(model.intercept_), float(model.coef_[0])

# =========================
# Main
# =========================
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"File not found: {INPUT_PATH}")

df = pd.read_csv(INPUT_PATH)

required = [GROUP_COL, X_COL, MEAN_TARGET, PEAK_TARGET]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}. Available: {df.columns.tolist()}")

# Ensure numeric
df[X_COL] = pd.to_numeric(df[X_COL], errors="coerce")
df[MEAN_TARGET] = pd.to_numeric(df[MEAN_TARGET], errors="coerce")
df[PEAK_TARGET] = pd.to_numeric(df[PEAK_TARGET], errors="coerce")

df = df.dropna(subset=required).copy()
df = df.sort_values([GROUP_COL, X_COL])

rows = []

for interval_id, g in df.groupby(GROUP_COL):
    x = g[X_COL].to_numpy(dtype=float)

    y_mean = g[MEAN_TARGET].to_numpy(dtype=float)
    y_peak = g[PEAK_TARGET].to_numpy(dtype=float)

    # ----- scratch -----
    s_mean_theta0, s_mean_theta1 = fit_scratch_with_centering(x, y_mean, LEARNING_RATE, ITERATIONS)
    s_peak_theta0, s_peak_theta1 = fit_scratch_with_centering(x, y_peak, LEARNING_RATE, ITERATIONS)

    # ----- sklearn -----
    k_mean_theta0, k_mean_theta1 = fit_sklearn(x, y_mean)
    k_peak_theta0, k_peak_theta1 = fit_sklearn(x, y_peak)

    rows.append({
        "interval_id": interval_id,
        "n_points": len(g),

        "scratch_mean_theta0": s_mean_theta0,
        "scratch_mean_theta1": s_mean_theta1,
        "scratch_peak_theta0": s_peak_theta0,
        "scratch_peak_theta1": s_peak_theta1,

        "sklearn_mean_theta0": k_mean_theta0,
        "sklearn_mean_theta1": k_mean_theta1,
        "sklearn_peak_theta0": k_peak_theta0,
        "sklearn_peak_theta1": k_peak_theta1,
    })

theta_table = pd.DataFrame(rows).sort_values("interval_id").reset_index(drop=True)

os.makedirs(OUTPUT_DIR, exist_ok=True)
theta_table.to_csv(OUTPUT_PATH, index=False)

print(f"Saved interval theta table to: {OUTPUT_PATH}")
print(theta_table.head(20))

Saved interval theta table to: data/models\interval_theta_table_model_implement.csv
    interval_id  n_points  scratch_mean_theta0  scratch_mean_theta1  \
0             1        10             0.177840            -0.102788   
1             2        10            -0.840417             0.099498   
2             3        10             0.812069            -0.030792   
3             4        10             3.709370            -0.092001   
4             5        10            -6.638421             0.143098   
5             6        10            -7.841825             0.148661   
6             7        10            -1.328246             0.025953   
7             8        10             8.827536            -0.117293   
8             9        10            -2.426644             0.025784   
9            10        10           -12.771144             0.136762   
10           11        10            -7.055246             0.066677   
11           12        10             1.921729            -0.016

# Model Evaluation


In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# =========================
# Config
# =========================
DATA_PATH = "data/processed/RMBR4-2_export_processed.csv"
THETA_PATH = "data/models/interval_theta_table_model_implement.csv"

OUTPUT_DIR = "data/models"
PLOT_DIR = os.path.join(OUTPUT_DIR, "plots")
OUTPUT_PATH = os.path.join(OUTPUT_DIR, "interval_theta_table_model_evaluated.csv")

X_COL = "work_period"
GROUP_COL = "interval_id"

MEAN_TARGET = "mean_value_z"
PEAK_TARGET = "peak_value_z"

# =========================
# Helpers
# =========================
def predict(theta0, theta1, x):
    """h_theta(x) = theta0 + theta1 * x"""
    return theta0 + theta1 * x

def compute_metrics(y_true, y_pred):
    return {
        "rmse": float(np.sqrt(mean_squared_error(y_true, y_pred))),
        "mae": float(mean_absolute_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred)),
    }

def plot_regression(x, y, y_scratch, y_sklearn, interval_id, target_name):
    plt.figure(figsize=(7, 5))
    plt.scatter(x, y, label="Data", color="black")
    plt.plot(x, y_scratch, label="Scratch", linewidth=2)
    plt.plot(x, y_sklearn, label="Sklearn", linestyle="--", linewidth=2)

    plt.xlabel("Work Period")
    plt.ylabel(target_name)
    plt.title(f"Interval {interval_id} – {target_name}")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    fname = f"interval_{interval_id}_{target_name}.png"
    plt.savefig(os.path.join(PLOT_DIR, fname))
    plt.close()

# =========================
# Load data
# =========================
df = pd.read_csv(DATA_PATH)
theta_df = pd.read_csv(THETA_PATH)

required_cols = [GROUP_COL, X_COL, MEAN_TARGET, PEAK_TARGET]
df = df.dropna(subset=required_cols).copy()

os.makedirs(PLOT_DIR, exist_ok=True)

results = []

# =========================
# Evaluation per interval
# =========================
for _, row in theta_df.iterrows():
    interval_id = row["interval_id"]

    g = df[df[GROUP_COL] == interval_id].sort_values(X_COL)
    if len(g) < 2:
        continue

    x = g[X_COL].to_numpy(dtype=float)

    # ----- MEAN -----
    y_mean = g[MEAN_TARGET].to_numpy(dtype=float)

    y_mean_scratch = predict(
        row["scratch_mean_theta0"],
        row["scratch_mean_theta1"],
        x
    )
    y_mean_sklearn = predict(
        row["sklearn_mean_theta0"],
        row["sklearn_mean_theta1"],
        x
    )

    mean_metrics_scratch = compute_metrics(y_mean, y_mean_scratch)
    mean_metrics_sklearn = compute_metrics(y_mean, y_mean_sklearn)

    plot_regression(
        x, y_mean,
        y_mean_scratch, y_mean_sklearn,
        interval_id, "mean"
    )

    # ----- PEAK -----
    y_peak = g[PEAK_TARGET].to_numpy(dtype=float)

    y_peak_scratch = predict(
        row["scratch_peak_theta0"],
        row["scratch_peak_theta1"],
        x
    )
    y_peak_sklearn = predict(
        row["sklearn_peak_theta0"],
        row["sklearn_peak_theta1"],
        x
    )

    peak_metrics_scratch = compute_metrics(y_peak, y_peak_scratch)
    peak_metrics_sklearn = compute_metrics(y_peak, y_peak_sklearn)

    plot_regression(
        x, y_peak,
        y_peak_scratch, y_peak_sklearn,
        interval_id, "peak"
    )

    results.append({
        "interval_id": interval_id,

        # Mean – scratch
        "scratch_mean_rmse": mean_metrics_scratch["rmse"],
        "scratch_mean_mae": mean_metrics_scratch["mae"],
        "scratch_mean_r2": mean_metrics_scratch["r2"],

        # Mean – sklearn
        "sklearn_mean_rmse": mean_metrics_sklearn["rmse"],
        "sklearn_mean_mae": mean_metrics_sklearn["mae"],
        "sklearn_mean_r2": mean_metrics_sklearn["r2"],

        # Peak – scratch
        "scratch_peak_rmse": peak_metrics_scratch["rmse"],
        "scratch_peak_mae": peak_metrics_scratch["mae"],
        "scratch_peak_r2": peak_metrics_scratch["r2"],

        # Peak – sklearn
        "sklearn_peak_rmse": peak_metrics_sklearn["rmse"],
        "sklearn_peak_mae": peak_metrics_sklearn["mae"],
        "sklearn_peak_r2": peak_metrics_sklearn["r2"],
    })

# =========================
# Merge metrics into theta table
# =========================
metrics_df = pd.DataFrame(results)
final_df = theta_df.merge(metrics_df, on="interval_id", how="left")

final_df.to_csv(OUTPUT_PATH, index=False)

print(f"Saved evaluated table to: {OUTPUT_PATH}")
print(f"Plots saved to: {PLOT_DIR}")
print(final_df.head())

Saved evaluated table to: data/models\interval_theta_table_model_evaluated.csv
Plots saved to: data/models\plots
   interval_id  n_points  scratch_mean_theta0  scratch_mean_theta1  \
0            1        10             0.177840            -0.102788   
1            2        10            -0.840417             0.099498   
2            3        10             0.812069            -0.030792   
3            4        10             3.709370            -0.092001   
4            5        10            -6.638421             0.143098   

   scratch_peak_theta0  scratch_peak_theta1  sklearn_mean_theta0  \
0             0.258191            -0.154178             0.177840   
1            -0.934766             0.073145            -0.840417   
2             2.212510            -0.074561             0.812069   
3             5.002872            -0.124509             3.709370   
4            -5.626308             0.113802            -6.638421   

   sklearn_mean_theta1  sklearn_peak_theta0  sklearn_peak

### 🔹 After Session 1 (Homework)

- Refactor your notebook into **modular Python scripts**:  
  - `data_loader.py` – functions to load data from CSV, API, and DB.  
  - `preprocessing.py` – cleaning, normalization, train/test split.  
  - `model.py` – regression model implementations.  
  - `evaluation.py` – metrics, plots, reporting.  
- Ensure each module can run independently.  

💡 This will prepare you for **Session 2 (MLOps)**.  

---

### 🔹 Before Session 2: Preparing for MLOps

- Replicate the structure, files and resources that you developed during the **DataStreamVisualization_Workshop**
- Use it to organize this project into a folder structure like:

```txt
linear_regression_project/
│── data/
│   ├── raw/
│   ├── processed/
│── notebooks/
│   ├── EDA.ipynb
│   ├── linear_regression.ipynb
│── src/
│   ├── data_loader.py
│   ├── preprocessing.py
│   ├── model.py
│   ├── evaluation.py
│── configs/
│   ├── experiment_config.yaml
│── experiments/
│   ├── results.csv
│── requirements.txt
│── README.md
````

* Create a **YAML config file** with parameters:

  * Data source path/API endpoint/DB connection string
  * Learning rate, iterations, train/test split ratio
  * Feature to use as predictor

* Document how to run your scripts step-by-step.

---

### 🔹 During Session 2: MLOps Architecture

* Apply the **Robot PM MLOps design patterns**:

  * **Separation of concerns**: Each module is independent.
  * **Configuration-driven**: Experiments are parameterized by configs, not hard-coded values.
  * **Experiment tracking**: Save model performance metrics in `experiments/results.csv`.
  * **Reproducibility**: Ensure anyone can re-run your experiment with the same results.

* Discuss:

  * Why modularity matters for ML projects.
  * How config management avoids errors in scaling ML experiments.
  * How this workflow connects to real-world ML pipelines.

💡 **Deliverable during Session 2**:

* A structured project with modular code, configs, and experiment tracking.

---

### Alert Design Based on Raw Robot Force Data

Based on the raw robot force data aggregated at the work-period level, alert thresholds are designed for both mean force and peak force to identify potential system degradation and failure risks.

The analysis is performed over fixed detection intervals, each consisting of a predefined number of consecutive work periods.

Alert Logic

For each detection interval, a linear regression model is fitted using the work period index as the independent variable. The absolute value of the regression slope is used to quantify the trend strength within the interval.

Trend-based Trigger Condition

If the absolute value of the slope within a detection interval exceeds a predefined slope threshold, the interval is flagged for further inspection.

This condition indicates an abnormal increasing or decreasing trend in the force signal over time.

Aging Alert (Mean Force)

Within the start and end time of the detection interval, if:

the absolute slope of the mean force exceeds the slope threshold, and

the average mean force within the interval exceeds the designed mean force alert threshold,

then an aging alert is raised.

This alert suggests that the system may be experiencing gradual degradation and may require maintenance.

Failure Alert (Peak Force)

Within the start and end time of the detection interval, if:

the absolute slope of the peak force exceeds the slope threshold, and

the maximum peak force within the interval exceeds the designed peak force alert threshold,

then a fault alert is raised.

This alert indicates the possibility of an imminent or acute system failure.

Interpretation

Mean force trends are used to capture long-term structural changes and gradual wear, making them suitable indicators for system aging.

Peak force trends are more sensitive to sudden stress or abnormal operating conditions and are therefore used to detect potential faults.

By combining trend-based detection with level-based thresholds, the alerting mechanism reduces false positives while maintaining sensitivity to meaningful system changes.

## Rationale for Selecting Mean and Peak Alert Thresholds

The alert thresholds for mean force and peak force are derived directly from the raw work-period level data to ensure data-driven and robust decision boundaries.

Mean Force Alert Threshold

The mean force alert threshold is designed to represent the upper bound of normal operating behavior under typical working conditions.

The threshold is calculated using a robust statistical approach based on the median and the Median Absolute Deviation (MAD):

Mean Alert Threshold
=
median(mean_value)
+
𝑘
×
1.4826
×
MAD(mean_value)
Mean Alert Threshold=median(mean_value)+k×1.4826×MAD(mean_value)

This method is preferred over mean and standard deviation because:

Robot force data often exhibit skewed distributions and outliers.

The median and MAD are less sensitive to extreme values, providing a more stable and reliable threshold.

The resulting threshold reflects the typical variability of the system rather than being dominated by rare extreme events.

When the average mean force within a detection interval exceeds this threshold, it suggests progressive mechanical wear or system aging, which is typically characterized by sustained increases in average load.

Peak Force Alert Threshold

The peak force alert threshold is designed to detect abnormal transient loads that may indicate imminent or sudden system failures.

The threshold is calculated using the same robust statistical framework:

Peak Alert Threshold
=
median(peak_value)
+
𝑘
×
1.4826
×
MAD(peak_value)
Peak Alert Threshold=median(peak_value)+k×1.4826×MAD(peak_value)

This approach is appropriate because:

Peak force signals are inherently more volatile and prone to sharp spikes.

Robust statistics prevent a small number of extreme spikes from inflating the threshold.

The threshold captures the upper range of normal transient behavior while remaining sensitive to abnormal stress events.

Exceeding the peak force alert threshold within a detection interval indicates a potential fault condition, such as mechanical jamming, collision, or actuator malfunction.

Choice of the Scaling Factor 
𝑘
k

The scaling factor 
𝑘
k controls the strictness of the alert thresholds.

A typical value of 
𝑘
=
3.5
k=3.5 is selected to:

Minimize false positives under normal operating conditions.

Preserve sensitivity to genuinely abnormal behavior.

This choice is consistent with established practices in robust outlier detection and industrial condition monitoring.

Summary

Mean force thresholds target long-term degradation and aging effects.

Peak force thresholds target short-term abnormal stress and fault conditions.

Both thresholds are data-driven, robust, and interpretable, making them suitable for real-world predictive maintenance applications.

In [14]:
import os
import numpy as np
import pandas as pd

# =========================
# Config
# =========================
PERIOD_PATH = "data/raw/RMBR4-2_export_test_1.csv"              # period-level summary with times
THETA_PATH  = "data/models/interval_theta_table_model_implement.csv"           # implement theta table (per interval)

OUTPUT_RESULTS_PATH = "experiments/results.csv"

INTERVAL_SIZE = 10
K = 3.5
PREDICT_OFFSET_DAYS = 14

# Choose which slope source to use: "sklearn" or "scratch"
SLOPE_SOURCE = "sklearn"

# =========================
# Robust threshold helpers
# =========================
def robust_location_scale(x: np.ndarray):
    """
    Returns (median, robust_sigma) where robust_sigma ~= std using MAD.
    robust_sigma = 1.4826 * MAD
    """
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    if len(x) == 0:
        return np.nan, np.nan
    med = np.median(x)
    mad = np.median(np.abs(x - med))
    robust_sigma = 1.4826 * mad
    return float(med), float(robust_sigma)

def robust_threshold(x: np.ndarray, k: float):
    med, rs = robust_location_scale(x)
    if not np.isfinite(med) or not np.isfinite(rs):
        return np.nan
    return float(med + k * rs)

# =========================
# Load period-level data
# =========================
if not os.path.exists(PERIOD_PATH):
    raise FileNotFoundError(f"Period file not found: {PERIOD_PATH}")
if not os.path.exists(THETA_PATH):
    raise FileNotFoundError(f"Theta file not found: {THETA_PATH}")

period_df = pd.read_csv(PERIOD_PATH)

required_period_cols = [
    "work_period", "mean_value", "peak_value", "period_start_time", "period_end_time"
]
missing = [c for c in required_period_cols if c not in period_df.columns]
if missing:
    raise ValueError(f"Missing columns in period file: {missing}\nAvailable: {period_df.columns.tolist()}")

period_df["work_period"] = pd.to_numeric(period_df["work_period"], errors="coerce")
period_df["mean_value"]  = pd.to_numeric(period_df["mean_value"], errors="coerce")
period_df["peak_value"]  = pd.to_numeric(period_df["peak_value"], errors="coerce")
period_df["period_start_time"] = pd.to_datetime(period_df["period_start_time"], errors="coerce", utc=True)
period_df["period_end_time"]   = pd.to_datetime(period_df["period_end_time"], errors="coerce", utc=True)

period_df = period_df.dropna(subset=["work_period", "mean_value", "peak_value", "period_end_time"]).copy()
period_df = period_df.sort_values("work_period").reset_index(drop=True)

# Build interval_id from work_period
period_df["interval_id"] = ((period_df["work_period"] - 1) // INTERVAL_SIZE) + 1

# =========================
# Build interval summary (start/end time + levels)
# =========================
interval_summary = (
    period_df.groupby("interval_id")
    .agg(
        interval_start_time=("period_start_time", "min"),
        interval_end_time=("period_end_time", "max"),
        interval_mean_level=("mean_value", "mean"),  # average mean over periods in interval
        interval_peak_level=("peak_value", "max"),   # max peak over periods in interval
        n_periods=("work_period", "count"),
    )
    .reset_index()
    .sort_values("interval_id")
    .reset_index(drop=True)
)

# =========================
# Load theta table (implement output)
# =========================
theta_df = pd.read_csv(THETA_PATH)

if SLOPE_SOURCE == "scratch":
    mean_slope_col = "scratch_mean_theta1"
    peak_slope_col = "scratch_peak_theta1"
else:
    mean_slope_col = "sklearn_mean_theta1"
    peak_slope_col = "sklearn_peak_theta1"

required_theta_cols = ["interval_id", mean_slope_col, peak_slope_col]
missing = [c for c in required_theta_cols if c not in theta_df.columns]
if missing:
    raise ValueError(f"Missing columns in theta file: {missing}\nAvailable: {theta_df.columns.tolist()}")

theta_df["interval_id"] = pd.to_numeric(theta_df["interval_id"], errors="coerce")
theta_df[mean_slope_col] = pd.to_numeric(theta_df[mean_slope_col], errors="coerce")
theta_df[peak_slope_col] = pd.to_numeric(theta_df[peak_slope_col], errors="coerce")
theta_df = theta_df.dropna(subset=["interval_id"]).copy()

# =========================
# Threshold design (from raw period-level data + theta table)
# =========================
# Level thresholds from raw period-level distributions
mean_alert_threshold = robust_threshold(period_df["mean_value"].to_numpy(), K)
peak_alert_threshold = robust_threshold(period_df["peak_value"].to_numpy(), K)

# Slope thresholds from observed slopes across intervals
mean_slope_threshold = robust_threshold(np.abs(theta_df[mean_slope_col].to_numpy()), K)
peak_slope_threshold = robust_threshold(np.abs(theta_df[peak_slope_col].to_numpy()), K)

# =========================
# Merge interval_summary with slopes
# =========================
merged = interval_summary.merge(
    theta_df[["interval_id", mean_slope_col, peak_slope_col]],
    on="interval_id",
    how="left"
)

# =========================
# Alert logic
# =========================
merged["mean_slope_flag"] = merged[mean_slope_col].abs() > mean_slope_threshold
merged["peak_slope_flag"] = merged[peak_slope_col].abs() > peak_slope_threshold

merged["aging_alert"] = merged["mean_slope_flag"] & (merged["interval_mean_level"] > mean_alert_threshold)
merged["fault_alert"] = merged["peak_slope_flag"] & (merged["interval_peak_level"] > peak_alert_threshold)

def failure_type(row):
    if row["aging_alert"] and row["fault_alert"]:
        return "Aging + Fault"
    if row["fault_alert"]:
        return "Fault"
    if row["aging_alert"]:
        return "Aging"
    return "No Alert"

merged["failure_type"] = merged.apply(failure_type, axis=1)

# Predicted failure time = interval end time + 14 days
merged["predicted_failure_time"] = merged["interval_end_time"] + pd.Timedelta(days=PREDICT_OFFSET_DAYS)

# Optional: add a human-readable reason (good for reports)
def build_reason(r):
    reasons = []
    if r["aging_alert"]:
        reasons.append("Possible system aging: mean slope & mean level exceed thresholds.")
    if r["fault_alert"]:
        reasons.append("Possible failure: peak slope & peak level exceed thresholds.")
    return " | ".join(reasons) if reasons else "No alert."

merged["alert_reason"] = merged.apply(build_reason, axis=1)

# =========================
# Build and save experiments/results.csv
# =========================
results_df = merged[[
    "interval_id",
    "interval_start_time",
    "interval_end_time",
    "predicted_failure_time",
    "failure_type",
    "alert_reason",
]].sort_values("interval_id").reset_index(drop=True)

os.makedirs(os.path.dirname(OUTPUT_RESULTS_PATH), exist_ok=True)
results_df.to_csv(OUTPUT_RESULTS_PATH, index=False)

print(f"Saved experiment tracking results to: {OUTPUT_RESULTS_PATH}")
print("\n--- Thresholds used ---")
print({
    "K": K,
    "slope_source": SLOPE_SOURCE,
    "mean_alert_threshold": mean_alert_threshold,
    "peak_alert_threshold": peak_alert_threshold,
    "mean_slope_threshold": mean_slope_threshold,
    "peak_slope_threshold": peak_slope_threshold,
})
print("\n--- Preview results ---")
print(results_df.head(20))

Saved experiment tracking results to: experiments/results.csv

--- Thresholds used ---
{'K': 3.5, 'slope_source': 'sklearn', 'mean_alert_threshold': 4.310588304459788, 'peak_alert_threshold': 10.972072798187499, 'mean_slope_threshold': 0.28998019526948016, 'peak_slope_threshold': 0.29888605648519484}

--- Preview results ---
    interval_id              interval_start_time  \
0             1 2022-10-17 12:19:22.005000+00:00   
1             2 2022-10-17 12:35:51.838000+00:00   
2             3 2022-10-17 13:08:22.832000+00:00   
3             4 2022-10-17 13:20:15.677000+00:00   
4             5 2022-10-17 13:47:28.226000+00:00   
5             6 2022-10-17 13:56:36.935000+00:00   
6             7 2022-10-17 14:07:11.332000+00:00   
7             8 2022-10-17 14:16:34.799000+00:00   
8             9 2022-10-17 14:26:26.237000+00:00   
9            10 2022-10-17 14:36:42.017000+00:00   
10           11 2022-10-17 14:47:08.972000+00:00   
11           12 2022-10-17 14:55:59.959000+00:00 

### 🔹 After Session 2: Extension & Homework

0. **Submission Format**  
   - This activity is **to be submitted individually**. Each student must create and manage their own project repository.

1. **Workshop Replication**  
   - This workshop is modeled on the structure, files, and resources used in the **DataStreamVisualization_Workshop**.  
   - Your submission must replicate this style of organization and completeness.  

2. **Repository Submission Instructions**  
   - Create a **remote Git repository** named:  
     ```
     LinearRegressionArchitecture_Workshop
     ```
   - Once your repository is ready, send your instructor an email with the subject line:  
     ```
     Linear Regression Architecture Workshop
     ```
   - In the body of the email, paste the **full URL of your repository**, making sure it ends with the `.git` extension.  
     - ✅ Correct example: `https://github.com/username/LinearRegressionArchitecture_Workshop.git`  
     - ❌ Incorrect example: `https://github.com/username/LinearRegressionArchitecture_Workshop`

3. **Repository Requirements**  
   Your repository must contain:  
   - A **frozen version of the codebase** (no further modifications after submission).  
   - A `requirements.txt` file that lists all dependencies required to run your project.  
   - A `README.md` file that:  
     - Displays the title: **Linear Regression Architecture Workshop**.  
     - Describes the work completed in the workshop.  
     - Summarizes key design decisions.  

4. **Notebook Updates (RobotPM_MLOps.ipynb)**  
   - Open the notebook `RobotPM_MLOps.ipynb`.  
   - Update it so that it highlights all changes made to the original project architecture and files.  
   - Specifically, reference the lists provided in the notebook:  
     - **Recommended Additions**  
     - **Recommended Enhancements**  
     - **Breakdown examples** (from both design breakdown sections).  

5. **Expectations for Notebook Updates**  
   - You are **not required to fully implement** the changes and updates at this stage.  
   - Instead, create all **placeholders, stubs, and structure** needed to prepare the project for a future code review.  
   - Think of this as **project scaffolding** for the upcoming implementation sprint cycle, which will be executed in a future project.  

💡 **Final Deliverable**:  
- A complete GitHub repository named `LinearRegressionArchitecture_Workshop` with the required structure, files, and documentation.  
- An updated `RobotPM_MLOps.ipynb` notebook showing how the project architecture was extended and prepared for enhancements.  
- Email submission to the instructor containing the `.git` repository URL.  
